In [1]:
!pip install openpyxl

!pip install openpyxl

In [2]:
import os
import pathlib

# [1-2]
# 必要なライブラリのインポート
import random
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow with Keras.
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

from tensorflow.keras.optimizers import SGD


# ART
from art.defences.trainer import AdversarialTrainer
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import ProjectedGradientDescent

from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split

import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook


%matplotlib inline

### モデルの読み込み(通常モデル、耐性モデル)

In [3]:
X = "npk4sen-10"#モデルファイル名

Y = "test脆弱"


In [4]:
model = tf.keras.models.load_model("../../models/npk4/"+X+".h5")#Naturalモデルのロード

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


### サンプルの読み込み(通常画像で固定) 読み込むのはVAL用画像

In [5]:
x_val = np.load('../../input/gtsrb-german-traffic-sign/Instant/x_val.npy')
y_val = np.load('../../input/gtsrb-german-traffic-sign/Instant/y_val.npy')

x_test = np.load('../../input/gtsrb-german-traffic-sign/Instant/x_test.npy')
y_test = np.load('../../input/gtsrb-german-traffic-sign/Instant/y_test.npy')


y_val =  to_categorical(y_val, num_classes=43)#評価のためにOne-Hotに
#二回実行するとエラーになる
y_test =  to_categorical(y_test, num_classes=43)#評価のためにOne-Hotに



print("test_samples: \n型{0},\nshape{1}".format(type(x_val),x_val.shape,))
print("\ntest_label: \n型{0},\nshape{1}".format(type(y_val),y_val.shape,))#整数形式

print("test_samples: \n型{0},\nshape{1}".format(type(x_test),x_test.shape,))
print("\ntest_label: \n型{0},\nshape{1}".format(type(y_test),y_test.shape,))#整数形式

test_samples: 
型<class 'numpy.ndarray'>,
shape(15684, 30, 30, 3)

test_label: 
型<class 'numpy.ndarray'>,
shape(15684, 43)
test_samples: 
型<class 'numpy.ndarray'>,
shape(12630, 30, 30, 3)

test_label: 
型<class 'numpy.ndarray'>,
shape(12630, 43)


In [6]:
# モデルをART Keras Classifierでラップ。
# ARTのClassifierインスタンスを作成する（ここではTensorFlowのモデルを想定）

classifier = KerasClassifier(model=model, clip_values=(0,1))

attack = ProjectedGradientDescent(classifier,eps=(8/255), eps_step=(2/255), max_iter=10, batch_size=64)

attack_FGSM = FastGradientMethod(estimator=classifier, eps=(8/255),norm=np.inf)




In [7]:
adv_samples_val = attack.generate(x_val)
adv_samples_test = attack.generate(x_test)
adv_samples_FGSM = attack_FGSM.generate(x_test)



### 敵対的サンプルの作成

    以下評価
  

In [8]:
loss,accuracy = model.evaluate(x_val, y_val)
print('val set accuracy: ', accuracy * 100)
N_ac_val = accuracy


loss,accuracy = model.evaluate(adv_samples_val, y_val)
print('val set accuracy: ', accuracy * 100)
AEs_ac_val = accuracy


loss,accuracy = model.evaluate(x_test, y_test)
print('val set accuracy: ', accuracy * 100)
N_ac_test = accuracy


loss,accuracy = model.evaluate(adv_samples_test, y_test)
print('val set accuracy: ', accuracy * 100)
AEs_ac_test = accuracy

loss,accuracy = model.evaluate(adv_samples_FGSM, y_test)
print('val set accuracy: ', accuracy * 100)
AEs_ac_FGSM = accuracy

#loss,accuracy = model.evaluate(adv_samples_PGD20, y_test)
#print('val set accuracy: ', accuracy * 100)
#AEs_ac_PGD20 = accuracy

15684/15684 [==============================] - 1s 80us/sample - loss: 0.3600 - accuracy: 0.9264
val set accuracy:  92.63580441474915
15684/15684 [==============================] - 1s 75us/sample - loss: 1.1184 - accuracy: 0.6226
val set accuracy:  62.260901927948
12630/12630 [==============================] - 1s 88us/sample - loss: 0.5651 - accuracy: 0.8614
val set accuracy:  86.13618612289429
12630/12630 [==============================] - 1s 81us/sample - loss: 1.4581 - accuracy: 0.5212
val set accuracy:  52.121931314468384
12630/12630 [==============================] - 1s 73us/sample - loss: 1.1617 - accuracy: 0.5939
val set accuracy:  59.390342235565186


In [9]:
print(round(N_ac_val,4))
print(round(AEs_ac_val,4))
print(round(N_ac_test,4))
print(round(AEs_ac_test,4))
print(round(AEs_ac_FGSM,4))
#print(round(AEs_ac_PGD20,4))


0.9264
0.6226
0.8614
0.5212
0.5939


### 保存

In [10]:
# 画像データは0～1の範囲、正解ラベルは元のy_testと変わらないので保存しない
#np.save("../../input/gtsrb-german-traffic-sign/Instant/"+X+".npy", adv_samples_val)
#np.save("../../input/gtsrb-german-traffic-sign/Instant/"+X+"-test.npy", adv_samples_test)
#np.save("../../input/gtsrb-german-traffic-sign/Instant/FGSM/"+X+"-test.npy", adv_samples_FGSM)
#np.save("../../input/gtsrb-german-traffic-sign/Instant/PGD20/"+X+"-test.npy", adv_samples_PGD20)

In [11]:
sheet_name = X+","+Y #シートの名前の定義
model_name =X
sample_name =Y
test_samples = adv_samples_test
test_labels = np.load('../../input/gtsrb-german-traffic-sign/Instant/y_test.npy')
#val_label= np.argmax(val_label,axis = 1) #y_testはOneHot形式

In [12]:
count_label = []#サンプルのクラスごとの枚数を格納
for j in range(43):    
    count = 0
    for i in range(len(test_labels)):
        if test_labels[i] == j:
            count +=1
    count_label.append(count)
moto = []#モデルが間違えた画像の正解ラベルを格納
suiron = []#モデルが間違えた画像を何と推論したかを格納
wrong_index = []#モデルが推論を間違えているインデックス
right_index = []#モデルが正解したインデックス、以下あまり使わない
for i in range(len(test_samples)):
    Y_hat = model.predict(np.expand_dims(test_samples[i],0))
    if test_labels[i] != np.argmax(Y_hat):        
        moto.append(test_labels[i])
        suiron.append(np.argmax(Y_hat))
        wrong_index.append(i)
    else:
        right_index.append(i)

wrong_predict = [] #間違えた画像の分類モデルの推論結果を格納する
count_num = []#クラスごとの間違えた数を格納する
for j in range(43):    
    count = 0
    kari_suiron = []
    for i in range(len(moto)):
        if moto[i] == j:
            count +=1
            kari_suiron.append(suiron[i])
    count_num.append(count)
    wrong_predict.append(kari_suiron)

wrong_predict_count = []
for j in range(43):
    count_kari = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    for i in range(len(wrong_predict[j])):
        count_kari[wrong_predict[j][i]] += 1
    wrong_predict_count.append(count_kari)
#間違えた数/元の枚数で割合を出す
wariai = []
for i in range(43):
    y = count_num[i] / count_label[i]
    wariai.append(y)

def index_of_second_largest(arr):
    sorted_indices = sorted(range(len(arr)), key=lambda i: arr[i], reverse=True)
    result = [sorted_indices.index(i) + 1 for i in range(len(arr))]
    return result

# 例: [8, 9, 2, 4, 7]
result = index_of_second_largest(wariai)

In [13]:
wb = load_workbook("../../最終検証.xlsx")
wb.create_sheet(title=sheet_name,index=1)
sheet = wb[sheet_name]




sheet["A4"] = model_name

sheet['C7'] = 'クラス'
for i in range(43):
    sheet["C"+str(i+8)]=i

sheet['D7'] = '画像の枚数'#count_label
sheet['E7'] = '間違えた枚数'#count_num
sheet['F7'] = '間違えた割合'#wariai
sheet['G7'] = '順位'

#クラスごとの間違え先wrong_predict[20]


sheet['C7'] = 'クラス'
for i in range(43):
    sheet["C"+str(i+8)]=i

sheet['D7'] = '画像の枚数'#count_label
sheet['E7'] = '間違えた枚数'#count_num
sheet['F7'] = '間違えた割合'#wariai
sheet['G7'] = '順位'

for i in range(43):
    sheet["D"+str(i+8)]=count_label[i]
    sheet["E"+str(i+8)]=count_num[i]
    sheet["F"+str(i+8)]=wariai[i]
    sheet["G"+str(i+8)]=result[i]

sheet["L6"]="元クラス"
sheet["K7"]="先クラス"

sheet["B2"]="val"
sheet["G2"]="Test"
sheet["B3"]="Natural"
sheet["D3"]="AEs"
sheet["G3"]="Natural"
sheet["I3"]="AEs"
sheet["L2"]="FGSM"
sheet["L3"]="AEs"



sheet["B4"]= round(N_ac_val,4)
sheet["D4"]= round(AEs_ac_val,4)
sheet["G4"]= round(N_ac_test,4)
sheet["I4"]= round(AEs_ac_test,4)
sheet["L4"]= round(AEs_ac_FGSM,4)



for i in range(43):
    sheet["K"+str(i+8)]=i
for i in range(43):
    sheet.cell(row=7,column=(12+i)).value="クラス"+str(i)
for i in range(43):
    for j in range(43):
        sheet.cell(row=j+8,column=12+i).value=wrong_predict_count[i][j]
wb.save("../../最終検証.xlsx")

In [14]:
from IPython.display import HTML

HTML("""
    <audio autoplay="">
        <source src="./pointaccent.mp3"></source>
    </audio>
""")